In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [2]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Garv Khurana\OneDrive\Desktop\datasets\IMDB Dataset.csv",)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["sentiment"]=le.fit_transform(df["sentiment"])
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [4]:
x=list(df["review"])
y=list(df["sentiment"])
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=50,test_size=0.4)

In [5]:
x_train=tokenizer(x_train,padding=True,truncation=True,return_tensors="tf")
x_test=tokenizer(x_test,padding=True,truncation=True,return_tensors="tf")

In [7]:
import tensorflow
training_dataset=tensorflow.data.Dataset.from_tensor_slices((dict(x_train),y_train)).batch(16)
test_dataset=tensorflow.data.Dataset.from_tensor_slices((dict(x_test),y_test)).batch(16)

In [8]:
x_train["input_ids"][0]

<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([  101,  2023,  3185,  2001,  6659,  1012,  1012,  1012,  2129,
        2064,  8307,  2130,  2228,  2008,  2023,  3185,  2001,  2066,
        1996,  3924,  2067,  1999,  1996,  1000,  2204,  2214,  2420,
        1000,  2043,  5199, 19438, 18617,  2001,  2025,  2130,  1999,
        2009,  1012,  1998,  2000,  2191,  2477,  4788,  1010,  2027,
        2109, 15281,  2013,  2214, 16588,  2099,  5691,  1012,  2008,
        2015,  2074,  6659,  1012,  2053, 16588,  2099,  3185,  2003,
        3143,  2302,  5199, 19438, 18617,  1012,  1045,  2293,  2440,
        5302,  2239,  3152,  1010,  1998,  1045,  2031,  2042,  3666,
        2068,  2144,  1045,  2001,  1018,  2086,  2214,  1012,  1045,
        2031,  2042,  2083,  2673,  2027,  2031,  2589,  1998,  2023,
        3185,  2471,  2081,  2033,  4558,  2009,  1012,  2085,  1045,
        2288,  1037,  3232,  1997,  3210,  2000,  6039,  2061,  1045,
        2097,  2562,  2183,  1012,  2037,  2

In [9]:
import tensorflow
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=5e-5)
loss = tensorflow.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer,loss=loss, metrics=['accuracy'])

In [10]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(training_dataset,epochs=1)



   2/1875 [..............................] - ETA: 27:00:15 - loss: 2.4452 - accuracy: 0.4688